## Load Conversations

In [1]:
import sys
sys.path.append('.')

import pandas as pd
from utils import load_jsonl, dump_jsonl
# import numpy as np

In [2]:
def get_task1_conver(in_dir, col_label, skips=[]):
    conversations = load_jsonl(f"{in_dir}")
    
    
    def to_message_str(messages, users):
        s = []
        u = []
        for m in messages:
            if users[m['user_id']] == "SYS":
                s.append(m['text'])
            else:
                u.append(m['text'])
                
        return s, u
        
    newdata = []
    for row in conversations:
        row["messages"].sort(key=lambda x: x["date_created"], reverse=False)
        
        users = {}
        for m in row["messages"]:
            if m["user_id"] not in users:
#                 username = "USR"+str(len(users)+1) if len(users.keys())==0 else "SYS"
                username = "USR" if len(users.keys())==0 else "SYS"
                users[m["user_id"]] = username
                
#         if len(users)>2:
#             print("More than 1 users", len(users))
        
        
        messages = row["messages"]
        chunk_size = 100
        for i in range(0, len(messages), chunk_size):
            sub_messages = messages[i:i+chunk_size]
            s, u = to_message_str(sub_messages, users)
            
            if pd.isna(row[col_label]):
                continue
            
            if row[col_label] in skips:
                continue
                
            label = row[col_label]
                
            newdata.append({
                "user": u,
                "sys": s,
                "label": label,
                "nturn": len(sub_messages)
            })
        
#     n_val = int(len(newdata)*0.05)
#     n_test = n_val
    
#     test = newdata[0:n_test]
#     val = newdata[n_test:n_test+n_val]
#     train = newdata[n_test+n_val:]
    
    return pd.DataFrame(newdata)

In [3]:
df = get_task1_conver("./Task1/annotated_conersations.jsonl", "closeness", skips = [])
df.head()

Loaded 1234 records from ./Task1/annotated_conersations.jsonl


,user,sys,label,nturn
0,"[ดีๆวาย, เป็นยังไงบ้างช่วงนี้, รวยๆ เฮงๆ , เหม...","[ว่าไงปราง, เหมือนเดิม ขายของ นอนตื่น ขายของ ช...",1. Close,35
1,"[วันนี้เรามาพูดถึงเรื่องภาพยนตร์กัน, ปกฅิแล้วค...","[โอเคค่ะ, ที่บ้านค่ะ, ดูแทบทุกวันเลย, ดูได้ทุก...",1. Close,39
2,"[เธอ, สรุป อ้วนจะซื้อรถเมื่อไหร่, แล้วอ้วนดูรถ...","[อะไรหมู, กลางปี, ว่าจะเอา ativ ตัวใหม่, ว่าจ...",1. Close,40
3,"[มึง, แมวมึงเป็นไงกันบ้างอ่ะ, ตอนนี้มีกี่ตัวนะ...","[ว่า, เด็กๆหรอ, ซนปกติเลยมึง, 5ตัว, แค่บ้านกูน...",1. Close,35
4,"[อ้วน, อยากได้แมว, อยากเลี้ยง, เอาไว้ตอนตัวไม่...","[ไรหมู, เอาไปทำไมแมว, จะหาจากไหน, เดี๋ยวถามให้...",1. Close,39


In [4]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Close,551,551,551
2. Know each other,230,230,230
3. Don't know each other,435,435,435
4. Don't like each other,5,5,5


In [5]:
# df.groupby("label").count()

In [6]:
def get_task2_conver(in_dir, col_label, skips=[]):
    conversations = load_jsonl(f"{in_dir}")
        
    newdata = []
    for row in conversations:
        row["messages"].sort(key=lambda x: x["created_at"], reverse=False)
        
        users = {}
        for m in row["messages"]:
            if m["user_id"] not in users:
#                 username = "USR"+str(len(users)+1) if len(users.keys())==0 else "SYS"
                username = "USR" if len(users.keys())!=0 else "SYS"
                users[m["user_id"]] = username
                
        if len(users)>2:
            print("More than 1 users", len(users))
        
        
        
        messages = row["messages"]
        s = []
        u = []
        for m in messages:
            text = m['text'].replace("[USR]", "").replace("[URL]", "URL")
            if users[m['user_id']] == "SYS":
                s.append(text)
            else:
                u.append(text)
        
        label = row[col_label]
            
        if pd.isna(label):
            continue
        
        if label in skips:
            continue
                
        newdata.append({
            "user": u,
            "sys": s,
            "label": label,
            "nturn": len(messages)
        })
                
            
    n_val = int(len(newdata)*0.1)
    n_test = n_val
    
    test = newdata[0:n_test]
    val = newdata[n_test:n_test+n_val]
    train = newdata[n_test+n_val:]
    
    return pd.DataFrame(newdata)

In [7]:
df = get_task2_conver("./Task2/annotated/annotated.jsonl", "authority")
df.head()

Loaded 2486 records from ./Task2/annotated/annotated.jsonl


,user,sys,label,nturn
0,[ แตงโม ชวนแฟนไปค่ะ ไม่ใช่กระติกชวนไป กลับไปหา...,[ทบทวน\nงานคืนนั้น #กระติก ชวนแฟนพี่ตม.ไปด้วยน...,3. Not respect,3
1,[ รูปนี้จ้า ยัยน้องตัวเล็กของพี่มิว☀️🌻\n \n#...,[เนื่ิองจากวันนี้...เป็นวันครบรอบ 3 ปี\nมาเล่น...,1. Respect,3
2,[ แอบคิดเหมือนกัน้ลยค่ะว่าจะมีอีกรอบ],[เธรดนี้จะมาวิเคราะห์น้ำท่วมปี54 ที่จริงมันอาจ...,2. Normal,3
3,[ เนี่ยสิ่งที่กูพูด URL],[มาดิ้นควยไรชองมึง\nมึงอ่ะโชว์โง่ไอ้สัตว์ เสร่...,3. Not respect,3
4,"[ สรุปปวินนี่บล็อกคนด้วยมั้ย?, สรุป ที่น่าโมโ...",[แต่ปวินกับไพรวัลย์ก็ไม่เข้าใจนะเหมือนเจ้เกลีย...,2. Normal,5


In [8]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Respect,319,319,319
2. Normal,1661,1661,1661
3. Not respect,364,364,364


In [9]:
df = get_task1_conver("./Task3/annotated/annotated.jsonl", "closeness")
df.head()

Loaded 1221 records from ./Task3/annotated/annotated.jsonl


,user,sys,label,nturn
0,"[ดีๆวาย, เป็นยังไงบ้างช่วงนี้, รวยๆ เฮงๆ , เหม...","[ว่าไงปราง, เหมือนเดิม ขายของ นอนตื่น ขายของ ช...",1. Close,35
1,"[วันนี้เรามาพูดถึงเรื่องภาพยนตร์กัน, ปกฅิแล้วค...","[โอเคค่ะ, ที่บ้านค่ะ, ดูแทบทุกวันเลย, ดูได้ทุก...",2. Know each other,39
2,"[เธอ, สรุป อ้วนจะซื้อรถเมื่อไหร่, แล้วอ้วนดูรถ...","[อะไรหมู, กลางปี, ว่าจะเอา ativ ตัวใหม่, ว่าจ...",2. Know each other,40
3,"[มึง, แมวมึงเป็นไงกันบ้างอ่ะ, ตอนนี้มีกี่ตัวนะ...","[ว่า, เด็กๆหรอ, ซนปกติเลยมึง, 5ตัว, แค่บ้านกูน...",1. Close,35
4,"[อ้วน, อยากได้แมว, อยากเลี้ยง, เอาไว้ตอนตัวไม่...","[ไรหมู, เอาไปทำไมแมว, จะหาจากไหน, เดี๋ยวถามให้...",1. Close,39


In [10]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Close,462,462,462
2. Know each other,696,696,696
3. Don't know each other,52,52,52
4. Don't like each other,11,11,11


## Load Lexicons

In [11]:
from pythainlp.tokenize import word_tokenize

In [12]:
import json
with open("./PrivateSpace/thai-dictionary/RoyalInstituteDictionary/words.json", encoding="utf-8") as fin:
    raw = json.load(fin)
    thaidict_royal = set()
    for k in raw:
        thaidict_royal.update(raw[k])

In [13]:
lexicons_arr = load_jsonl("lexicons.jsonl")

Loaded 25573 records from lexicons.jsonl


In [14]:
from collections import defaultdict
tags = set()
lexicons = {}
lexicons_keys = defaultdict(list)

for key, values  in lexicons_arr:
    if len(key) <= 1:
        continue
        
    key = key.lower()
    if key.endswith("rep"):
        key = key.replace("rep", "")
        
    w = word_tokenize(key)
    
    lexicons_keys[w[0]].append(key)
    
    tag = [t for t in values["tags"] if not t.startswith("cat:")]
    lexicons[key] = tag
    tags.update(tag)

In [15]:
from pythainlp.util import countthai

In [16]:
import re
from itertools import groupby
import emoji
from pythainlp.util import countthai
from tqdm import tqdm

def rm_reptitive(text):
    s = ""
    groups = groupby(text)
    for label, group in groups:
        g = list(group)
        if len(g) >= 3:
            s += f"{label*3} rep "
        else:
            s += "".join(g)
    return s

def remove_space(sent):
    newwords = []
    for w in sent:
        if len(w.strip())==0:
            continue
        newwords.append(w)
    return newwords
    
def analyse_conv_per_person(texts):
    
    # Word Statistic
    texts = [t.lower() for t in texts]
    texts = [rm_reptitive(t) for t in texts]
    words = [word_tokenize(t) for t in texts]
    words = [remove_space(w) for w in words]
    
    nlongword = 0
    ndict = 0
    for sent in words:
        ndict += sum([1 if w in thaidict_royal else 0 for w in sent])
        nlongword += sum([1 if len(w) > 7 else 0 for w in sent])
        nthai = sum([1 if countthai(w) > 50 else 0 for w in sent])
        
        
    
    uwords = set()
    for sent in words:
        uwords.update(sent)
    
    # Lexicon 
    lex = []
    for sidx, sent in enumerate(words):
        for widx, w in enumerate(sent):
            if w not in lexicons_keys:
                continue 
            
            s = "".join(sent[widx:])
            for l in lexicons_keys[w]:
                if not s.startswith(l):
                    continue

                lex.extend(lexicons[l])
#                 print(">>", w, l, lexicons[l])

    lexcat = {}
    for l in lex:
        if l not in lexcat:
            lexcat[l] = 0
        lexcat[l] += 1
    
    # Stylistic words
    nrepeat = 0
    for sidx, sent in enumerate(words):
        nrepeat += sum([1 if w=="rep" else 0 for w in sent])
    
    s = " ".join(texts)
    nemoji = emoji.emoji_count(s)
    

    return {
        "nsent": len(texts),
        "nword": sum([len(w) for w in words]),
        "ndict": ndict,
        "nunique": len(uwords),
        "nlongword": nlongword,
        "nrepeat": nrepeat,
        "nthai": nthai,
        "nemoji": nemoji,
        **lexcat
    }
  


def analyse_conversation(df):
    metrics = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        ms = analyse_conv_per_person(row["sys"])
        mu = analyse_conv_per_person(row["user"])
        metrics.append((ms, mu))
    
    return metrics

analyse_conv_per_person(["เมิงงงงงงงมันโง่เหมือนควายยยยยยยยย"]);

## Run Analysis

In [17]:
import pickle
def save_obj_values(filepath, obj):
  with open(filepath, 'wb') as fin:
    pickle.dump(obj, fin)
    
def load_obj_values(filepath):
  with open(filepath, 'rb') as fin:
    obj = pickle.load(fin)
  return obj

In [18]:
analysis_values = {
    "closeness": [None, None, None],
    "authority": [None, None, None],
}

analysis_labels = {
    "closeness": [None, None, None],
    "authority": [None, None, None],
}

In [19]:
# df = get_task1_conver("./Task1/annotated_conersations.jsonl", "closeness", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["closeness"][0] = metrics
# analysis_labels["closeness"][0] = df["label"].values


# df = get_task2_conver("./Task2/annotated/annotated.jsonl", "closeness", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["closeness"][1] = metrics
# analysis_labels["closeness"][1] = df["label"].values


# df = get_task1_conver("./Task3/annotated/annotated.jsonl", "closeness", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["closeness"][2] = metrics
# analysis_labels["closeness"][2] = df["label"].values

In [20]:
# df = get_task1_conver("./Task1/annotated_conersations.jsonl", "authority", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["authority"][0] = metrics
# analysis_labels["authority"][0] = df["label"].values


# df = get_task2_conver("./Task2/annotated/annotated.jsonl", "authority", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["authority"][1] = metrics
# analysis_labels["authority"][1] = df["label"].values


# df = get_task1_conver("./Task3/annotated/annotated.jsonl", "authority", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["authority"][2] = metrics
# analysis_labels["authority"][2] = df["label"].values

In [21]:
# save_obj_values("analysis_values.pkl", analysis_values)
# save_obj_values("analysis_labels.pkl", analysis_labels)

In [22]:
analysis_values = load_obj_values("analysis_values.pkl")
analysis_labels = load_obj_values("analysis_labels.pkl")

In [277]:
metric_names = {
    "Conversational Statistics" : {
        "nsent": "Number of utterance",
        "nword": "Number of word",
    },
    "Linguistic Complexity" : {
        "nunique": "Unique lexicons",
        "nthai": "Code-switching",
        "nlongword": "Long words",
        "ndict": "Dictionary words",
        "transliterated": "Transliteration",
    },
    "Pronoun": {
#         "pronoun": "All pronoun",
        "pronoun_1st": ">> 1st person pronoun",
        "pronoun_2nd": ">> 2nd person pronoun",
        "pronoun_3rd": ">> 3rd person pronoun",
        "pronoun_misspelling": ">> Pronoun in non-standard spelling",
    },
    
    "Sentence-ending Particles": {
#         "pronoun": "All particles",
        "particles_SARP": ">> Socially-related particles",
        "particles_notSARP": ">> Non-socially-related particles",
        "particles_misspelling": ">> Particle in non-standard spelling",
    },
    
    "Sentiment-related": {
#         "sentiment": "Sentiment words",
        "sentiment_positive": ">> Positive words",
        "sentiment_negative": ">> Negative words",
    },
    
    "Spelling variation": {
#         "misspelling": "Spelling variation",
        "misspelling_common": ">> Common misspelt words",
        "misspelling_intention": ">> Morphophonemic variation",
        "misspelling_shorten": ">> Simplified variation",
        "nrepeat": ">> Repeated characters",
    },
    "Other Common Internet Lexicons": {
        "abbr": "Abbreviation",
#         "slang": "Slang",
        "swear": "Swear words"
    }
}

In [278]:
clse_print_labels = ['1. Close', '2. Know each other', "3. Don't know each other", "4. Don't like each other"]
auth_print_labels = ['0. Very respect', '1. Respect', '2. Normal', '3. Not respect']

## Linear Regression

In [279]:
def to_features(analysis_values, analysis_labels, factor, setting, to_vec_func, skips=[]):
    values = analysis_values[factor][setting]
    labels = analysis_labels[factor][setting]
    assert(len(values)==len(labels))
    
    rows = []
    for (ms, mu), l in zip(values, labels):
        if l in skips:
            continue
        
        x = {}
        for m in mu:
            if m in ["nsent", "nword"]:
                continue
            x[m] = mu[m]*100/mu["nword"]
        
        if "particles_SARP" in x:
            x["particles_notSARP"] = x["particles"] - x["particles_SARP"]
                
        y = to_vec_func(l)
        rows.append({
            **x,
            **y
        })
            
    feats = pd.DataFrame(rows)
    feats.dropna(subset=["y"], inplace=True)
    feats.drop(columns=['particles_??', "nemoji", "slang"], inplace=True)  ## drop unused features
    feats.drop(columns=['sentiment', "misspelling", "particles", "pronoun"], inplace=True) ## drop overal features
    feats.fillna(0, inplace=True)
    return feats

In [280]:
import statsmodels.api as sm
import numpy as np

def print_anova_test(feats):
    x_columns = [col for col in feats.columns if col!="y"]
    X = feats[x_columns]
    X = sm.add_constant(X)
    Y = feats["y"]
    model = sm.OLS(Y,X)
    results = model.fit()

    
    ncoef = len(x_columns)+1
    
    t_test = results.t_test(np.identity(ncoef))
    f_test = results.f_test(np.identity(ncoef))
    
    weights = []
    for i, col in enumerate(["bias"]+x_columns):
        o = {
            "feat": col, 
            "coef": t_test.effect[i], 
            "pval": t_test.pvalue[i]
        }

        weights.append(o)
    
    outputs = pd.DataFrame(weights)
    return outputs

In [281]:
# def print_weights(outputs, coef_labels, labels, base_category=None):
    
#     coefs = {}
#     non_coefs = {}
#     for i, label in enumerate(coef_labels):
#         if i==0:
#             continue
            
#         cond = outputs[f"pval{i}"] < 0.05
#         for _, row in outputs[cond].sort_values(f"coef{i}", ascending=False).iterrows():
#             coefs[(label, row["feat"])] = row[f"coef{i}"]
        
#         cond = outputs[f"pval{i}"] >= 0.05
#         for _, row in outputs[cond].iterrows():
#             non_coefs[(label, row["feat"])] = row[f"coef{i}"]

# #     print(coefs)
#     for g in metric_names:
#         print("\multicolumn{5}{l}{\\textit{"+g+"}} \\\\")
#         for m in metric_names[g]:
#             s = f"{metric_names[g][m]} "
#             for l in labels:
#                 if l == base_category:
#                     s += f"& * "
#                 elif (l, m) in coefs:
#                     val = coefs[(l, m)]
#                     s += "& \cellcolor{gray!25} "+f"{val:.2f} "
#                 elif (l, m) in non_coefs:
#                     val = non_coefs[(l, m)]
#                     s += f"& {val:.2f} "
#                 else:
#                     s += f"& - "

#             s += "\\\\"
#             print(s)
#         print("&  & &  & \\\\")
#         print("\hline")

# # gray!25 < gray!50 < gray!80

In [282]:
def closeness_to_vec1(label):
    if label == "1. Close":
        return {"y": 1}
    elif label == "2. Know each other":
        return {"y": 0.5}
    elif label == "3. Don't know each other": 
        return {"y": 0}
    elif label == "4. Don't like each other":
        return {}
    else:
        return {}

feats = to_features(analysis_values, analysis_labels, "closeness", 0, closeness_to_vec1, skips=[])
clos1_weights = print_anova_test(feats)
clos1_weights[clos1_weights["pval"]<0.05]

,feat,coef,pval
2,nunique,0.008239,9.589083e-08
3,nlongword,-0.009592,3.403139e-02
4,nrepeat,-0.017450,4.691556e-03
5,nthai,-0.002816,1.038987e-02
10,pronoun_3rd,0.029463,6.240134e-06
13,particles_SARP,-0.015084,3.821109e-16
14,particles_notSARP,-0.022674,1.357549e-09
18,misspelling_common,0.019376,4.922897e-02


In [283]:
feats.columns

Index(['ndict', 'nunique', 'nlongword', 'nrepeat', 'nthai',
       'sentiment_positive', 'misspelling_shorten', 'misspelling_intention',
       'pronoun_2nd', 'pronoun_3rd', 'pronoun_1st', 'sentiment_negative', 'y',
       'particles_SARP', 'particles_notSARP', 'transliterated',
       'particles_misspelling', 'abbr', 'misspelling_common',
       'pronoun_misspelling', 'swear'],
      dtype='object')

In [284]:
def authority_to_vec1(label):
    if label == '0. Very respect':
        return {"y":1}
    elif label == '1. Respect': 
        return {"y":0.5}
    elif label == '2. Normal':
        return {"y":0}
    elif label == '3. Not respect':
        return {}
    else:
        return {}

feats = to_features(analysis_values, analysis_labels, "authority", 0, authority_to_vec1, skips=["3. Not respect"])
auth1_weights = print_anova_test(feats)
auth1_weights[auth1_weights["pval"]<0.05]

,feat,coef,pval
0,bias,0.513089,0.021404
2,nunique,-0.005109,0.000595
4,nrepeat,0.018699,0.002118
5,nthai,0.002102,0.048193
8,misspelling_intention,-0.004532,0.040372
9,pronoun_2nd,-0.010224,0.037646
12,sentiment_negative,-0.011951,0.028246
14,particles_notSARP,0.007933,0.024519
20,swear,0.030411,0.042311


In [285]:
feats = to_features(analysis_values, analysis_labels, "closeness", 1, closeness_to_vec1, skips=[])
clos2_weights = print_anova_test(feats)
clos2_weights[clos2_weights["pval"]<0.05]

,feat,coef,pval
1,ndict,-0.001105,2.485390e-02
2,nunique,0.002773,1.318220e-04
4,nrepeat,0.009112,6.427944e-07
7,misspelling_intention,0.002114,1.106582e-02
15,sentiment_positive,0.003068,8.544853e-03
20,swear,0.035813,1.042942e-08


In [286]:
def authority_to_vec2(label):
    if label == '0. Very respect':
        return {}
    elif label == '1. Respect': 
        return {"y":1}
    elif label == '2. Normal':
        return {"y":0.5}
    elif label == '3. Not respect':
        return {"y":0}
    else:
        return {}

feats = to_features(analysis_values, analysis_labels, "authority", 1, authority_to_vec2)
auth2_weights = print_anova_test(feats)
auth2_weights[auth2_weights["pval"]<0.05]

,feat,coef,pval
0,bias,0.765414,1.460918e-64
1,ndict,-0.003231,6.637271e-21
4,nrepeat,-0.002480,2.882226e-02
6,misspelling_shorten,-0.002634,1.611300e-05
7,misspelling_intention,0.002382,2.199503e-05
8,particles_SARP,0.006336,3.157573e-09
9,sentiment_negative,-0.006481,1.458833e-12
10,sentiment_positive,0.006552,4.455281e-16
11,particles_notSARP,0.005035,3.105326e-03
17,pronoun_misspelling,-0.005534,3.538780e-02


In [287]:
feats = to_features(analysis_values, analysis_labels, "closeness", 2, closeness_to_vec1, skips=[])
clos3_weights = print_anova_test(feats)
clos3_weights[clos3_weights["pval"]<0.05]

,feat,coef,pval
0,bias,0.479821,5.318368e-04
2,nunique,0.003898,3.062918e-05
3,nlongword,-0.009839,3.495579e-04
5,nthai,-0.002172,1.141511e-03
6,sentiment_positive,-0.003576,2.052889e-02
10,pronoun_3rd,0.016256,4.871385e-05
11,pronoun_1st,0.012241,1.680557e-03
12,sentiment_negative,0.010343,2.716573e-03
13,particles_SARP,-0.008501,3.433633e-14
14,particles_notSARP,-0.009353,3.550289e-05


In [288]:
feats = to_features(analysis_values, analysis_labels, "authority", 2, authority_to_vec2)
auth3_weights = print_anova_test(feats)
auth3_weights[auth3_weights["pval"]<0.05]

,feat,coef,pval
0,bias,0.625286,3.949011e-11
3,nlongword,0.005317,4.376929e-03
4,nrepeat,-0.007839,2.063218e-03
5,nthai,0.002225,9.519263e-07
7,misspelling_shorten,-0.003570,9.866583e-04
11,pronoun_1st,-0.005934,2.098490e-02
13,particles_SARP,0.003519,3.235055e-06
17,abbr,-0.011162,2.693347e-02
19,pronoun_misspelling,-0.008525,2.873819e-02
20,swear,-0.015337,1.369579e-02


In [289]:
# clos3_weights

### Print Results

In [290]:
sections = [
    "Setting 1: Private Conversations with Self-Reported Labels",
    "Setting 2: Public Conversations with Labels from 3rd Party ",
    "Setting 3: Private Conversations with Labels from 3rd Party ",
]

In [291]:
def outputs_to_dict(outputs):
    coefs = {}
    
    for _, row in outputs.iterrows():
        coefs[row["feat"]] = (row["coef"], row["pval"])

    return coefs

In [292]:
printed_text = "" 
printed_text += "\subsection{Closeness}"+"\n"
outputs = [
    outputs_to_dict(clos1_weights),
    outputs_to_dict(clos2_weights),
    outputs_to_dict(clos3_weights)
]

printed_text += "\\begin{longtable}[h]{"+"\n"
printed_text += "    p{\dimexpr 0.40\linewidth-2\\tabcolsep}|"+"\n"

printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"
printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"
printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"

printed_text += "}"+"\n"
printed_text += "    \hline"+"\n"
printed_text += "    Lexical Features & Setting 1  & Setting 2 & Setting 3 \\\\"+"\n"
printed_text += "    \hline"+"\n"
#     printed_text += "    \endfirsthead"+"\n"
#     printed_text += ""+"\n"
printed_text += "    \endhead"+"\n"
printed_text += ""+"\n"

# for sec, results in zip(sections, outputs):
for g in metric_names:
    if g in ["Conversational Statistics"]:
        continue

    printed_text += "    \multicolumn{4}{l}{\\textit{"+g+"}} \\\\"+"\n"
    printed_text += "    \hline"+"\n"

    for m in metric_names[g]:
        s = f"        {metric_names[g][m]} "
        for out in outputs:
            if m not in out:
                s += f"& - "
            else:
                val, pval = out[m]

                if pval < 0.05:
                    s += "& \cellcolor{gray!25} "+f"{val:.2e} "
                else:
                    s += f"& {val:.2e} "
        s += "\\\\"
        # print(s)
        printed_text += s+"\n"


    printed_text += "        &  & &\\\\"+"\n"
    printed_text += "    \hline"+"\n"
    printed_text += ""+"\n"
printed_text += "\label{closeness_linear_weights}"+"\n"
printed_text += "\end{longtable}"+"\n"
printed_text += "\clearpage"+"\n"



In [293]:
printed_text += "\subsection{Respect}"+"\n"
outputs = [
    outputs_to_dict(auth1_weights),
    outputs_to_dict(auth2_weights),
    outputs_to_dict(auth3_weights)
]

printed_text += "\\begin{longtable}[h]{"+"\n"
printed_text += "    p{\dimexpr 0.40\linewidth-2\\tabcolsep}|"+"\n"

printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"
printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"
printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"

printed_text += "}"+"\n"
printed_text += "    \hline"+"\n"
printed_text += "    Lexical Features & Setting 1  & Setting 2 & Setting 3 \\\\"+"\n"
printed_text += "    \hline"+"\n"
#     printed_text += "    \endfirsthead"+"\n"
#     printed_text += ""+"\n"
printed_text += "    \endhead"+"\n"
printed_text += ""+"\n"

# for sec, results in zip(sections, outputs):
for g in metric_names:
    if g in ["Conversational Statistics"]:
        continue

    printed_text += "    \multicolumn{4}{l}{\\textit{"+g+"}} \\\\"+"\n"
    printed_text += "    \hline"+"\n"

    for m in metric_names[g]:
        s = f"        {metric_names[g][m]} "
        for out in outputs:
            if m not in out:
                s += f"& - "
            else:
                val, pval = out[m]

                if pval < 0.05:
                    s += "& \cellcolor{gray!25} "+f"{val:.2e} "
                else:
                    s += f"& {val:.2e} "
        s += "\\\\"
        # print(s)
        printed_text += s+"\n"


    printed_text += "        &  & &\\\\"+"\n"
    printed_text += "    \hline"+"\n"
    printed_text += ""+"\n"
printed_text += "\label{closeness_linear_weights}"+"\n"
printed_text += "\end{longtable}"+"\n"



In [304]:
print(printed_text)

\subsection{Closeness}
\begin{longtable}[h]{
    p{\dimexpr 0.40\linewidth-2\tabcolsep}|
    p{\dimexpr 0.15\linewidth-2\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\tabcolsep}
}
    \hline
    Lexical Features & Setting 1  & Setting 2 & Setting 3 \\
    \hline
    \endhead

    \multicolumn{4}{l}{\textit{Linguistic Complexity}} \\
    \hline
        Unique lexicons & \cellcolor{gray!25} 8.24e-03 & \cellcolor{gray!25} 2.77e-03 & \cellcolor{gray!25} 3.90e-03 \\
        Code-switching & \cellcolor{gray!25} -2.82e-03 & -9.27e-05 & \cellcolor{gray!25} -2.17e-03 \\
        Long words & \cellcolor{gray!25} -9.59e-03 & -1.00e-03 & \cellcolor{gray!25} -9.84e-03 \\
        Dictionary words & -3.01e-03 & \cellcolor{gray!25} -1.10e-03 & -8.73e-04 \\
        Transliteration & -7.26e-03 & -2.52e-03 & -9.70e-03 \\
        &  & &\\
    \hline

    \multicolumn{4}{l}{\textit{Pronoun}} \\
    \hline
        >> 1st person pronoun & 1.11e-02 & 5.16e-03 & \cellcolo

### Evaluate Linear Model

In [308]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from random import random

def get_data_from_df_to_data(df):
    x_columns = [col for col in df.columns if col!="y"]
    X = df[x_columns]
    X = sm.add_constant(X)
    Y = df["y"]
    
    return X, Y
    
def train_model(feats, random_state=42, random_guess=False):
    d = feats.sample(frac=1, random_state=random_state)
    train, test = np.split(d, [int(0.9*len(d))])
#     train, validate, test = np.split(d, [int(0.8*len(d)), int(0.9*len(d))])
    
    Xtrain, Ytrain = get_data_from_df_to_data(train)
    model = sm.OLS(Ytrain,Xtrain)
    M = model.fit()
    
#     Xval, Yval = get_data_from_df_to_data(validate)
#     thesholds = tune_thesholds(M, Xval, Yval)
        
    Xtest, Ytest = get_data_from_df_to_data(test)
    if random_guess:
        predictions = [random() for y in Ytest]
    else:
        predictions = M.predict(Xtest)
    
    return np.sqrt(mean_squared_error(Ytest, predictions)), r2_score(Ytest, predictions)

In [331]:
def print_accuracy(feats, random_guess=False):
    rmse = []
    r2 = []
    for i in range(100):
        _rmse, _r2 = train_model(feats, random_state=i, random_guess=random_guess)
        rmse.append(_rmse)
        r2.append(_r2)

    print("RMSE", np.mean(rmse))
    print("R2", np.mean(r2))

In [339]:
random_guess=False

In [340]:
feats = to_features(analysis_values, analysis_labels, "closeness", 0, closeness_to_vec1, skips=[])
print_accuracy(feats, random_guess=random_guess)

RMSE 0.40490237548091335
R2 0.17288599460124274


In [341]:
feats = to_features(analysis_values, analysis_labels, "authority", 0, authority_to_vec1, skips=[])
print_accuracy(feats, random_guess=random_guess)

RMSE 0.39518452510090185
R2 0.01315226793270619


In [342]:
feats = to_features(analysis_values, analysis_labels, "closeness", 1, closeness_to_vec1, skips=[])
print_accuracy(feats, random_guess=random_guess)

RMSE 0.3306880896803744
R2 0.034903721621206335


In [343]:
feats = to_features(analysis_values, analysis_labels, "authority", 1, authority_to_vec2)
print_accuracy(feats, random_guess=random_guess)

RMSE 0.24797473375388374
R2 0.15318322578028065


In [344]:
feats = to_features(analysis_values, analysis_labels, "closeness", 2, closeness_to_vec1, skips=[])
print_accuracy(feats, random_guess=random_guess)

RMSE 0.24653696225942773
R2 0.20312443854909148


In [345]:
feats = to_features(analysis_values, analysis_labels, "authority", 2, authority_to_vec2)
print_accuracy(feats, random_guess=random_guess)

RMSE 0.16542602154762986
R2 0.1188264316478763
